In [102]:
import csv
import requests
from bs4 import BeautifulSoup


years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
years = [2020, 2021]  # Det finns ett tak på antal celler, så det går bara att ta ett par år i taget

region_data = []
with open("regions.csv", "r") as file:
    reader = csv.DictReader(file)
    region_data = [r for r in reader]

regions = [x["code"] for x in region_data]

formdata = {
    'vTABELL': '2', # Biståndsmotagare
    'vMATT': ';6;;7;', # antal och andel mottagare
    'hvOMR': "".join([";{};".format(r) for r in regions]),
    'vAR': "".join([";{};".format(y) for y in years]),
    'vMANAD': "".join([";{};".format(m) for m in range(1, 13)]),
    'vKON': ";3;;2;;1;",  # Män, kvinnor och total
    # Dessa används inte i sökningen, men måste ändå vara med
    'vUTRIKES': '', 'vHINDER': '', 'vGRUPP_ANTAL_BARN': '', 'vPERSORDNGRP': '', 'vHUSHALLSTYP': '', 'vAGI': '',
    # Krävs av någon anledning...
    'clientScreenWidth': '900',
    'clientScreenHeight': '600',
}


url = 'https://sdb.socialstyrelsen.se/if_ekb_manad/resultat.aspx'
r = requests.post(url, formdata)

soup = BeautifulSoup(r.text, "html.parser")
table = soup.find("table", {'id': "ph1_GridView1"})

data = []
measures = {
    'Andel vuxna biståndsmottagare av befolkning': "rate",
    'Antal vuxna biståndsmottagare': "count",
}
genders = {
    'Män': "male",
    'Kvinnor': "female",
    'Båda könen': "all",
}
rows = table.find_all("tr")[2:]  # de två första raderna är kolumnrubriker i den här sökningen
for row in rows:
    cells = row.find_all("td")
    cells = [c.text for c in cells]
    current_region = cells.pop(0)
    region_data_ = [r for r in region_data if r["socialstyrelsen"] == current_region][0]

    obj = {
        'region': region_data_["newsworthy"],
        'region_type': region_data_["type"],
        'cldr': region_data_["cldr"],
        'measure': measures[cells.pop(0)],
        'gender': genders[cells.pop(0)],
        'note': "",
    }
    # en kolumn per månad
    # TODO: kontrollera mot kolumnrubrik att vi verkligen hämtar rätt månad
    for y in years:
        for m in months:
            _ = obj.copy()
            _["timepoint"] = "{}-{}-01".format(y, str(m).zfill(2))
            val = cells.pop(0).replace("\xa0", "").replace(",", ".")
            if val == "--":
                # inget värde rapporterat än
                continue
            if val == "X":
                _["value"] = None
                _["note"] = "Värde saknas pga statistiksekretess."
            elif _["measure"] == "count":
                _["value"] = int(val)
            else:
                _["value"] = float(val)
            data.append(_)

with open("output.csv", "w") as file_:
    writer = csv.DictWriter(file_, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)
